In [14]:
%%capture
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained(
    "openai/clip-vit-base-patch32")

2024-03-09 12:41:22.818176: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-09 12:41:25.298305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 12:41:25.298704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 12:41:26.239269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 12:41:26.988949: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from PIL import Image
image = Image.open("cat.jpg")
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)

In [3]:
from video_processor import VideoProcessor

In [5]:
from transformers import AutoImageProcessor, VideoMAEModel
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

In [7]:
vp = VideoProcessor(
    clip_model.text_model,
    model,
    processor.tokenizer,
    image_processor,
    device="cpu",
)
video,audio = vp.extract_frames_and_audio(
    "/DATA/sarmistha_2221cs21/basha/VideoMAE/dataset_224x224/_gPgueccVzw.mp4"
)

Extracting frame chunks


100%|██████████| 10908/10908 [04:24<00:00, 41.28it/s]
/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:139: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(video_path, sr=16000)
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Extracting audio chunks


100%|██████████| 188/188 [00:00<00:00, 211742.52it/s]


In [8]:
import pickle
with open("frames_audio__gPgueccVzw.pkl",'wb') as f:
    pickle.dump([video,audio],f)

In [14]:
encoding = vp.vision_encoder(video)

Encoding frames


/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


In [18]:
import os
os.getcwd()
video_dir = '/home/basha_2211ai03/complaint_detection/videos/dataset'
videos = os.listdir(video_dir)

In [17]:
processor.tokenizer('basha',return_tensors = 'pt')

{'input_ids': tensor([[49406,   932,  3337, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [22]:
import pickle
try:
    with open("data.pkl",'rb') as f:
        encoded_data = pickle.load(f)
except:
    encoded_data = {}

In [23]:
len(encoded_data)

5

In [25]:

import torch
import pickle
from tqdm.notebook import tqdm
with torch.inference_mode():
    for video in tqdm(videos):
        if video in encoded_data:
            continue
        out = vp(os.path.join(video_dir,video))
        encoded_data[video] = out
        with open("data.pkl",'wb') as f:
            pickle.dump(encoded_data,f)

  0%|          | 0/408 [00:00<?, ?it/s]

NoBackendError: 

In [6]:
import pickle
with open("data.pkl",'rb') as f:
    out = pickle.load(f)

In [9]:
vp = vp.eval()

In [19]:
processor.tokenizer('basha',return_tensors = 'pt')

{'input_ids': tensor([[49406,   932,  3337, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [15]:
outputs.last_hidden_state.mean(1).squeeze().shape

torch.Size([768])

In [16]:
clip_model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [1]:
import pickle
with open("data.pkl",'rb') as f:
    dataset = pickle.load(f)

In [2]:
len(dataset)

346

In [3]:
dataset['Lb0mjQ50hkg.mp4'][0].shape

torch.Size([194, 512])

In [4]:
from models import DetectSpan
detect_span_model = DetectSpan(512)

In [5]:
detect_span_model  = detect_span_model.cuda()

In [6]:
dataset['Lb0mjQ50hkg.mp4'][0].shape

torch.Size([194, 512])

In [28]:
from typing import List
import torch

def pad(batch: List[torch.tensor], dim: int = 512, max_len: int = None) -> torch.tensor:
    if max_len is None:
        max_len = max([len(x) for x in batch])
    attention_mask = []
    for i in range(len(batch)):
        ones = [1] * len(batch[i])
        pad_length = max_len - len(batch[i])
        if pad_length == 0:
            attention_mask.append(ones)
            continue
        zeros = [0] * pad_length
        padding = torch.zeros(pad_length, dim)
        batch[i] = torch.cat([batch[i], padding ])
        attention_mask.append(ones + zeros)
    return torch.stack(batch), torch.tensor(attention_mask)

In [27]:
processor.tokenizer(['basha','i'],return_tensors='pt',padding = True)

{'input_ids': tensor([[49406,   932,  3337, 49407],
        [49406,   328, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}

In [29]:
dataset['Lb0mjQ50hkg.mp4'][0].shape

torch.Size([194, 512])

In [30]:
dataset['VBebt51tOUE.mp4'][0].shape

torch.Size([83, 512])

In [31]:
out = pad([dataset['Lb0mjQ50hkg.mp4'][0].cpu(),dataset['VBebt51tOUE.mp4'][0].cpu()])

In [95]:
import torch
import torch.nn as nn
from transformers import BertConfig, BertModel

torch.manual_seed(10)


class DetectSpan(nn.Module):
    """Model to detect span in a video

    Args:
        input_shape (int): input shape to the model
    """

    def __init__(self, input_shape, nhead=12, d_model=768,num_blocks = 12):
        """Init method for DetectSpan."""
        super().__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True),
            num_layers=num_blocks,
        )
        self.project = nn.Linear(input_shape,d_model)
        self.classification_layer = nn.Linear(d_model, 3)
        self.sigmoid = nn.Sigmoid()
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, labels=None):
        x = self.project(x)
        x = self.encoder(x, src_key_padding_mask=attention_mask.to(torch.bool))
        x = self.classification_layer(x)
        x = self.sigmoid(x)
        if labels:
            return self.loss(x, labels)
        return x

    def loss(self, model_outputs, labels):
        self.loss_function(model_outputs, labels)

In [96]:
detect_span_model = DetectSpan(512)

In [97]:
detect_span_model = detect_span_model.cuda()

In [98]:
detect_span_model

DetectSpan(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (project): Linear(in_features=512, out_features=768, bias=True)
  (classification_layer): Linear(in_features=768, out_features=3, bias=True)
  (sigmoid): Sigmoid()
  (loss_function): CrossEntropyLoss()
)

In [94]:
with torch.no_grad():
    masked = detect_span_model(out[0].cuda(), attention_mask=out[1].cuda())

tensor([[[ 0.1375,  0.0839,  0.0598,  ..., -0.1668, -0.1738,  0.1406],
         [ 0.1133,  0.1077,  0.0212,  ..., -0.2659, -0.1425,  0.0915],
         [ 0.1456,  0.1448, -0.0016,  ..., -0.1683, -0.1475,  0.1722],
         ...,
         [ 0.0587, -0.1146,  0.0255,  ..., -0.1927, -0.1174,  0.1000],
         [ 0.0732,  0.0676, -0.0312,  ..., -0.0893, -0.5749,  0.0433],
         [ 0.0810, -0.0746, -0.0896,  ..., -0.1709, -0.5200, -0.0180]],

        [[-0.2935,  0.3012, -0.0762,  ...,  0.0495, -0.2012, -0.0411],
         [-0.2836,  0.2760, -0.1742,  ..., -0.0578, -0.2003,  0.0056],
         [-0.1369,  0.1105, -0.4463,  ..., -0.1642, -0.1567,  0.1483],
         ...,
         [ 0.0253,  0.0211,  0.0414,  ...,  0.0421,  0.0236,  0.0368],
         [ 0.0253,  0.0211,  0.0414,  ...,  0.0421,  0.0236,  0.0368],
         [ 0.0253,  0.0211,  0.0414,  ...,  0.0421,  0.0236,  0.0368]]],
       device='cuda:0')
tensor([[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,   

In [90]:
masked[1]

tensor([[0.6030, 0.7292, 0.6452],
        [0.6370, 0.7084, 0.6756],
        [0.6359, 0.6429, 0.6601],
        [0.6455, 0.6558, 0.6613],
        [0.5199, 0.6695, 0.6153],
        [0.6228, 0.6745, 0.6378],
        [0.6518, 0.6133, 0.6677],
        [0.6133, 0.6674, 0.6346],
        [0.6342, 0.6678, 0.6494],
        [0.6464, 0.6610, 0.6514],
        [0.6231, 0.6689, 0.6556],
        [0.5927, 0.6622, 0.6374],
        [0.5849, 0.6476, 0.6199],
        [0.6048, 0.6513, 0.6388],
        [0.6283, 0.6886, 0.6677],
        [0.6119, 0.6781, 0.6486],
        [0.6021, 0.6478, 0.6356],
        [0.6602, 0.6594, 0.6226],
        [0.5874, 0.6849, 0.6599],
        [0.6031, 0.6417, 0.6729],
        [0.6053, 0.6799, 0.6258],
        [0.6020, 0.7039, 0.6235],
        [0.5559, 0.6898, 0.6699],
        [0.6259, 0.6686, 0.6573],
        [0.6373, 0.6943, 0.6354],
        [0.5863, 0.6953, 0.6746],
        [0.6482, 0.7212, 0.6743],
        [0.5855, 0.6682, 0.7138],
        [0.5938, 0.6614, 0.6784],
        [0.609

In [38]:
from transformers import BertConfig, BertModel
encoder = BertModel(BertConfig())

In [52]:
outs = processor.tokenizer(['basha','sha'],return_tensors = 'pt',padding = True)

In [58]:
encoder

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [24]:
processor.tokenizer('bas',return_tensors = 'pt')

{'input_ids': tensor([[49406, 11420, 49407]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:

out = clip_model.text_model(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

In [ ]:
out.last_hidden_state.shape

In [ ]:
out.pooler_output

In [ ]:
out.last_hidden_state[0]